https://youtu.be/0ZqeBEa_MWo?si=euGFINmBNnY8qPbn&t=149

In [ ]:
import copy

import holoviews as hv
import numpy as np
import pandas as pd
from icecream import ic

In [ ]:
hv.extension("bokeh")

In [ ]:
spool_radius = 5.75
gear_ratio = (59.0 / 17.0) ** 2
STEP_DIVISION = 1
motor_steps_per_revolution = 100 * STEP_DIVISION
spool_circumfrence = spool_radius * 2 * np.pi
ic(spool_circumfrence)
steps_per_mm = motor_steps_per_revolution * gear_ratio / spool_circumfrence
ic(steps_per_mm)
max_rpm = 100.0
max_revs_per_second = max_rpm / 60.0
max_steps_per_second = max_revs_per_second * motor_steps_per_revolution
ic(max_steps_per_second)
max_velocity = max_steps_per_second / steps_per_mm
ic(max_velocity)
max_acceleration = 1.0
max_jerk = 1.0

In [ ]:
def s_curve(max_d):
    """Solve minimal s curve that hits parameters

    max_d: [0: max_velocity, 1: max_acceleration, 2: max_jerk]
    """
    nd = len(max_d)
    assert nd == 3
    ns = 7
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]
    sc = [{"t":0.0, "dt": 0.0, "x": np.zeros(nd + 1), "dx": np.zeros(nd + 1)} for i in range(ns)]
    jerk_stage_sign = [1, 0, -1, 0, -1, 0, 1]
    coast_stage = 3

    dt_jerk = min(max_d[1] / max_d[2], np.sqrt(max_d[0] * 2 / max_d[2]))
    dv_jerk = 1 / 2 * max_d[2] * dt_jerk ** 2
    dt_acceleration = (max_d[0] - dv_jerk * 2) / max_d[1]

    for(i, dt) in enumerate([dt_jerk, dt_acceleration, dt_jerk, 0.0, dt_jerk, dt_acceleration, dt_jerk]):
        sc[i]["dt"] = dt
        if i == 0:
            t_prev = 0
        else:
            t_prev = sc[i - 1]["t"]
        sc[i]["t"] = t_prev + sc[i]["dt"]
    
    for i in range(ns):
        if i == 0:
            x_prev = [0.0 for _ in range(nd + 1)]
        else:
            x_prev = sc[i - 1]["x"]
        # set d_jerk
        sc[i]["dx"][3] = max_d[2] * jerk_stage_sign[i]
        # set d_acceleration
        sc[i]["dx"][2] = sc[i]["dx"][3] * sc[i]["dt"]
        # set d_velocity
        sc[i]["dx"][1] = x_prev[2] * sc[i]["dt"] + 1 / 2 * sc[i]["dx"][3] * sc[i]["dt"] ** 2
        # set d_position
        sc[i]["dx"][0] = x_prev[1] * sc[i]["dt"] + 1 / 2 * x_prev[2] * sc[i]["dt"] ** 2 + 1 / 6 * sc[i]["dx"][3] * sc[i]["dt"] ** 3
        # set x
        sc[i]["x"] = x_prev + sc[i]["dx"]
    return sc
    # todo: use this to generate both full curve and minimal, 4 stage curve
    # make sanity check function

In [ ]:
def s_curve_long(sc, dt):
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]
    nx = len(sc[0]["x"])
    state = {"t": 0.0, "x": np.zeros(nx), "p": 0.0, "v": 0.0, "a": 0.0, "j": 0.0}
    curve = [state.copy()]
    for i in range(len(sc)):
        if i > 0:
            state["x"] = sc[i - 1]["x"].copy()
        while state["t"] < sc[i]["t"]:
            state["t"] += dt
            state["x"][2] += sc[i]["dx"][3] * dt
            state["x"][1] += state["x"][2] * dt
            state["x"][0] += state["x"][1] * dt
            state["p"] = state["x"][0]
            state["v"] = state["x"][1]
            state["a"] = state["x"][2]
            state["j"] = state["x"][3]
            curve.append(state.copy())
        error = np.abs(state["x"][0] - sc[i]["x"][0])
        assert error / dt < 10
    
    return curve

In [ ]:
sc_f = s_curve([max_velocity, max_acceleration, max_jerk])

In [ ]:
sc_long = s_curve_long(sc_f, 0.001)

In [ ]:
hv.Curve(sc_long, "t", "p") + hv.Curve(sc_long, "t", "v") + hv.Curve(sc_long, "t", "a") + hv.Curve(sc_long, "t", "j")

In [ ]:
start = np.array([150.0, 200.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
def full_s_curve(start, end):
    """Solve for full s-curve and ignore joint constraints"""
    
    dist = np.linalg.norm(start - end)
    
    if dist < sc[6]["p"]:
        print("path too short")
    
    out = copy.deepcopy(sc)
    
    # update coast stage (3) to fill up remaining time
    out[3]["dp"] = dist - sc[6]["p"]
    out[3]["dt"] = out[3]["dp"] / out[3]["v"]
    for i in range(3,7):
        out[i]["p"] = out[i-1]["p"] + out[i]["dp"]
        out[i]["t"] = out[i-1]["t"] + out[i]["dt"]
        
    return out

In [ ]:
out = full_s_curve(start, end)

In [ ]:
hv.Curve(pd.DataFrame(out), "t", "v") + hv.Curve(pd.DataFrame(out), "t", "p")

# Shorter?

Ok that's great. But what happens if we want to go shorter than a full curve? The next version would be one with truncated full acceleration stages (1 and 5)

Make a medimum s curve

# Minimal Curve

We first need to solve the constant jerk only version of the curve with no coasting or max accleration stages

In [ ]:
sc_min = [{"t":0.0, "v":0.0, "p":0.0, "dt": 0.0, "dv": 0.0, "dp": 0.0} for i in range(4)]

In [ ]:
# either max_jerk to max_acceleration or max_velocity
sc_min[0]["dt"] = sc_min[0]["t"] = min(ic(max_acceleration / max_jerk), ic(np.sqrt(max_velocity * 2 / max_jerk)))
sc_min[0]["dv"] = sc_min[0]["v"] = max_jerk * sc_min[0]["t"] ** 2 / 2
sc_min[0]["dp"] = sc_min[0]["p"] = max_jerk * sc_min[0]["t"] ** 3 / 6
ic(sc_min[0])

In [ ]:
# dt is the same for all sc_min segments
for i in range(1,4):
    sc_min[i]["dt"] = sc_min[0]["dt"]

# dv changes sign
sc_min[1]["dv"] = sc_min[0]["dv"]
sc_min[2]["dv"] = sc_min[3]["dv"] = -sc_min[0]["dv"]

# add up dv and dt to get v and t
for i in range(1,4):
    sc_min[i]["v"] = sc_min[i - 1]["v"] + sc_min[i]["dv"]
    sc_min[i]["t"] = sc_min[i - 1]["t"] + sc_min[i]["dt"]

# compute values of dp

# stage 1 dp
# j = -max_jerk
# a = max_acceleration - max_jerk * t
# v = sc_min[0]["v"] + max_acceleration * t - 1 / 2 *max_jerk * t ** 2
# p = sc_min[0]["p"] + sc_min[0]["v"] * t + 1/2 * max_acceleration * t ** 2 - 1/6 * max_jerk * t ** 3

sc_min[1]["dp"] = sc_min[0]["v"] * sc_min[1]["dt"] + 1 / 2 * max_acceleration * sc_min[1]["dt"] ** 2 - 1 / 6 * max_jerk * sc_min[1]["dt"] ** 3

# stage 2 dp
# j = -max_jerk
# a = -max_jerk * t
# v = sc[1]["v"] - 1 / 2 * max_jerk * t ** 2
# p = sc[1]["p"] + sc[1]["v"] * t - 1 / 6 * max_jerk * t ** 3

sc_min[2]["dp"] = sc_min[1]["v"] * sc_min[2]["dt"] - 1 / 6 * max_jerk * sc_min[2]["dt"] ** 3

sc_min[3]["dp"] = sc_min[2]["v"] * sc_min[3]["dt"] - 1 / 2 * max_acceleration * sc_min[3]["dt"] ** 2 + 1 / 6 * max_jerk * sc_min[3]["dt"] ** 3


for i in range(1, 4):
    sc_min[i]["p"] = sc_min[i - 1]["p"] + sc_min[i]["dp"]

ic(sc_min)

In [ ]:
hv.Curve(sc_min, "t", "v") + hv.Curve(sc_min, "t", "p")

# Sanity check minimal curve

In [ ]:
# section 0
dt = 0.00001
state = {
    "t": 0,
    "v": 0,
    "a": 0,
    "p": 0
}
min_hist = [state.copy()]
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[0]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[0])
ic(sc_min[0]["p"] - min_hist[-1]["p"])

In [ ]:
# section 1
state = {
    "t": sc_min[0]["t"],
    "v": sc_min[0]["v"],
    "a": max_acceleration,
    "p": sc_min[0]["p"]
}
while True:
    state["t"] += dt
    state["a"] += -max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[1]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[1])
ic(sc_min[1]["p"] - min_hist[-1]["p"])

In [ ]:
# section 2
state = {
    "t": sc_min[1]["t"],
    "v": sc_min[1]["v"],
    "a": 0,
    "p": sc_min[1]["p"]
}
while True:
    state["t"] += dt
    state["a"] -= max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[2]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[2])
ic(sc_min[2]["p"] - min_hist[-1]["p"])

In [ ]:
# section 3
state = {
    "t": sc_min[2]["t"],
    "v": sc_min[2]["v"],
    "a": -max_acceleration,
    "p": sc_min[2]["p"]
}
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[3]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[3])
ic(sc_min[3]["p"] - min_hist[-1]["p"])

In [ ]:
hv.Curve(min_hist, "t", "v") + hv.Curve(min_hist, "t", "p")

In [ ]:
start = np.array([15.0, 20.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
dist = np.linalg.norm(start - end)

In [ ]:
upper_thresh = sc[6]["p"]
lower_thresh = sc[0]["dp"] * 4

In [ ]:
if dist > sc[6]["p"] || dist < 